In [1]:
import pandas as pd
from datetime import datetime
df = pd.read_csv('research_traffic.csv')
print(df.shape)
print(df['Label'].value_counts()[1])
print(df['Label'].value_counts()[0])
df.head()
# df = df.iloc[[40665]]
# df.head()
print(df.iloc[[40665]])

(140000, 12)
70000
70000
       Count_of_Source_IP  Port_Count  Pair_Count_Ratio  Packet_Count_Diff  \
40665                 545         545               0.0                  1   

       Lookup_Count_Diff  Protocol  Average_Packet_Count  Average_Byte_Count  \
40665                  0        17                   0.0                 0.0   

       Packet_Std_Dev  Byte_Std_Dev  Duration_per_Flow  Label  
40665             0.0           0.0           0.365574      1  


In [2]:
import pickle
import numpy as np
from scipy.stats import mode
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold, cross_validate
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, classification_report
)
from datetime import datetime

y = df.Label
X = df.drop(['Label'],axis=1)

# Normalize the dataset using MinMaxScaler
normalizer = MinMaxScaler()
X_normalized = normalizer.fit_transform(X)
X = pd.DataFrame(X_normalized, columns=X.columns)
print("Dataset normalized using MinMaxScaler.")

X_train_cv, X_unseen_test, y_train_cv, y_unseen_test = train_test_split(X, y, test_size=0.2, random_state=42)

kf = KFold(n_splits=10, shuffle=True, random_state=42)
print("Data normalized and split.")

Dataset normalized using MinMaxScaler.
Data normalized and split.


In [3]:
models = []
start = datetime.now()
pickle_files = ["ann_binary.pkl", "knn_binary.pkl", "xgb_binary.pkl", "dt_binary.pkl", "nb_binary.pkl", "lgb_binary.pkl", "lr_binary.pkl", "rf_binary.pkl", "svm_binary.pkl"]
for filename in pickle_files:
    with open(filename, 'rb') as file:
        model = pickle.load(file)
    model_name = filename.replace("_binary.pkl", "")
    models.append((model_name, model))
print(f"{len(models)} models loaded successfully.")

9 models loaded successfully.


In [4]:
predictions = {}
for name, model in models:
    if name == "ann":
        prediction = np.array((model.predict(X_unseen_test) > 0.5).astype(int))
    else:
        prediction = np.array(model.predict(X_unseen_test))
    if prediction.ndim > 1:
        prediction = prediction.ravel()
    predictions[name] = prediction

    acc = accuracy_score(y_unseen_test, prediction)
    prec = precision_score(y_unseen_test, prediction)
    rec = recall_score(y_unseen_test, prediction)
    f1  = f1_score(y_unseen_test, prediction)
    cm  = confusion_matrix(y_unseen_test, prediction)

    # Print the results for the current model
    print(f"Results for model: {name}")
    print("Accuracy: ", acc)
    print("Precision:", prec)
    print("Recall:   ", rec)
    print("F1 Score: ", f1)
    print("Confusion Matrix:")
    print(cm)
    print("-" * 40)
# # Optional: Inspect one of the predictions
# print(predictions['ann'])
# Step 2: Combine predictions and apply majority vote ensemble using pandas mode
# Create a DataFrame where each column corresponds to a model's predictions
pred_df = pd.DataFrame(predictions)
pred_df.head()

print("Predictions obtained from all models.")

# For each sample (row), get the mode (most common value) across the classifiers.
# Since there are 9 classifiers (an odd number), ties should not occur.
ensemble_predictions = pred_df.mode(axis=1)[0].values

# Step 3: Evaluate the ensemble predictions using standard metrics
accuracy  = accuracy_score(y_unseen_test, ensemble_predictions)
precision = precision_score(y_unseen_test, ensemble_predictions)
recall    = recall_score(y_unseen_test, ensemble_predictions)
f1        = f1_score(y_unseen_test, ensemble_predictions)
cm        = confusion_matrix(y_unseen_test, ensemble_predictions)

# Print out the results
print("Majority Vote Ensemble Evaluation (Pandas Mode):")
print("Accuracy: ", accuracy)
print("Precision:", precision)
print("Recall:   ", recall)
print("F1 Score: ", f1)
print("Confusion Matrix:\n", cm)

875/875 ━━━━━━━━━━━━━━━━━━━━ 1s 940us/step
Results for model: ann
Accuracy:  0.9839285714285714
Precision: 0.9761391304347826
Recall:    0.9924322795105736
F1 Score:  0.9842182787402679
Confusion Matrix:
[[13518   343]
 [  107 14032]]
----------------------------------------
Results for model: knn
Accuracy:  0.98275
Precision: 0.9770821688093907
Recall:    0.9890374142442888
F1 Score:  0.9830234438156831
Confusion Matrix:
[[13533   328]
 [  155 13984]]
----------------------------------------
Results for model: xgb
Accuracy:  0.9868571428571429
Precision: 0.9785252623531865
Recall:    0.9958271447768583
F1 Score:  0.9871003925967471
Confusion Matrix:
[[13552   309]
 [   59 14080]]
----------------------------------------
Results for model: dt
Accuracy:  0.9785357142857143
Precision: 0.978442182640656
Recall:    0.9790649975245774
F1 Score:  0.9787534910029342
Confusion Matrix:
[[13556   305]
 [  296 13843]]
----------------------------------------
Results for model: nb
Accuracy:  0.931

In [5]:
from deap import base, creator, tools, algorithms
from sklearn.metrics import accuracy_score

# Define the objective function to maximize accuracy
def evaluate(weights):
    weighted_predictions = []
    for i in range(len(y_unseen_test)):
        class_votes = {}
        for j, (name, model) in enumerate(models):
            pred = predictions[name][i]
            class_votes[pred] = class_votes.get(pred, 0) + weights[j]
        weighted_predictions.append(max(class_votes, key=class_votes.get))
    acc = accuracy_score(y_unseen_test, weighted_predictions)
    return (acc,)


In [6]:
# Set up the Genetic Algorithm
num_models = len(models)
creator.create("FitnessMax", base.Fitness, weights=(1.0,))  # Maximize accuracy
creator.create("Individual", list, fitness=creator.FitnessMax)

# Define population and individual initialization
toolbox = base.Toolbox()
toolbox.register("attr_float", np.random.rand)  # Random weight initialization

toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=num_models)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Genetic operators
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.2, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

# Run Genetic Algorithm
pop = toolbox.population(n=50)  # Population size
algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=50, verbose=True)

# Extract best weights
best_individual = np.array(tools.selBest(pop, k=1)[0])
optimized_weights = best_individual / np.sum(best_individual)  # Normalize weights

# Store classifier names with optimized weights in a dictionary
classifier_names = [name for name, _ in models]
classifier_weights = {clf_name: weight for clf_name, weight in zip(classifier_names, optimized_weights)}

print("Optimized Classifier Weights:", classifier_weights)

gen	nevals
0  	50    
1  	35    
2  	20    
3  	33    
4  	35    
5  	30    
6  	30    
7  	29    
8  	32    
9  	33    
10 	29    
11 	32    
12 	25    
13 	29    
14 	25    
15 	18    
16 	24    
17 	30    
18 	31    
19 	40    
20 	30    
21 	34    
22 	34    
23 	32    
24 	31    
25 	27    
26 	29    
27 	31    
28 	28    
29 	35    
30 	27    
31 	30    
32 	16    
33 	28    
34 	27    
35 	30    
36 	29    
37 	34    
38 	30    
39 	41    
40 	29    
41 	42    
42 	23    
43 	28    
44 	30    
45 	37    
46 	24    
47 	27    
48 	29    
49 	38    
50 	28    
Optimized Classifier Weights: {'ann': 0.0007125540412833424, 'knn': 0.11223164100871337, 'xgb': 0.32267916146757, 'dt': 0.35360769610060067, 'nb': -0.05292927734128819, 'lgb': -0.06984599326363714, 'lr': -0.2580602524956523, 'rf': 0.5236689154112776, 'svm': 0.06793555507113266}


In [7]:
classifier_weights = {'ann': 0.04337417816855901, 'knn': 0.005602240325772464, 'xgb': 0.4703796361375013, 'dt': 0.436420021927582, 'nb': -0.35000903676225836, 'lgb': 0.03877736424779778, 'lr': -0.14712137662935784, 'rf': 0.6058174300103625, 'svm': -0.10324045742595897}

In [8]:
# List to store the final weighted ensemble predictions
weighted_ensemble_predictions = []
n_samples = len(y_unseen_test)

# Loop over each sample in the test set
for i in range(n_samples):
    weighted_votes = {}

    for name, prediction in predictions.items():
        pred_class = prediction[i]
        weight = classifier_weights.get(name, 1.0)
        # Accumulate the vote for the predicted class
        weighted_votes[pred_class] = weighted_votes.get(pred_class, 0) + weight

    final_class = max(weighted_votes, key=weighted_votes.get)
    weighted_ensemble_predictions.append(final_class)

end = datetime.now()

accuracy  = accuracy_score(y_unseen_test, weighted_ensemble_predictions)
precision = precision_score(y_unseen_test, weighted_ensemble_predictions)
recall    = recall_score(y_unseen_test, weighted_ensemble_predictions)
f1        = f1_score(y_unseen_test, weighted_ensemble_predictions)
cm        = confusion_matrix(y_unseen_test, weighted_ensemble_predictions)


print("Weighted Ensemble Evaluation:")
print("Accuracy: ", accuracy)
print("Precision:", precision)
print("Recall:   ", recall)
print("F1 Score: ", f1)
print("Confusion Matrix:\n", cm)
print("training time: ", end-start)

Weighted Ensemble Evaluation:
Accuracy:  0.98775
Precision: 0.9798942535132878
Recall:    0.9961807765754297
F1 Score:  0.9879703994669098
Confusion Matrix:
 [[13572   289]
 [   54 14085]]
training time:  0:05:45.296030
